<a href="https://colab.research.google.com/github/karthikm-0/csc2515-project-stackoverflow/blob/main/csc2515_project_stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import drive
drive.mount('/content/drive')
projec_folder = "./drive/My Drive/csc2515-project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load and Preprocess Data**

In [ ]:
import pandas as pd
import os
import numpy as np

questions = pd.read_csv('/content/drive/My Drive/csc2515-project/Questions.csv',  encoding='Latin-1')
questions.head(5)
answers = pd.read_csv('/content/drive/My Drive/csc2515-project/Answers.csv',  encoding='Latin-1')
answers.head(5)

In [ ]:
questions.columns=["QId", "QAskerId", "QCreationDate", "QClosedDate", "QScore", "QTitle", "QBody"]
questions.head(5)

,QId,QAskerId,QCreationDate,QClosedDate,QScore,QTitle,QBody
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
#Count number of answers for questions with answers
counts = answers['ParentId'].value_counts()
#Perform an inner join on question id between questions and answers for questions with 7 or more answers
questions_answers = pd.merge(questions,answers[answers['ParentId'].isin(counts.index[counts >=7])], left_on=['QId'], right_on=['ParentId'])
samples=questions_answers.shape[0]
print("Total samples numbers with 7+ answers:",samples)

Total samples numbers with 7+ answers: 120435


In [ ]:
#Generate label for each answer, with True indicating the best answer
labels = questions_answers.groupby(['QId'])['Score'].transform(max) == questions_answers['Score']

**MING FEATURE SELECTION**

In [ ]:
#Find number of answers before and after
before=np.zeros((samples,1))
after=np.zeros((samples,1))
#Answers are already sorted by ID/date
before=questions_answers.groupby(['QId']).cumcount()
after=(questions_answers[::-1].groupby(['QId']).cumcount())[::-1]
print(before)
print(after)

0         0
1         1
2         2
3         3
4         4
         ..
120430    2
120431    3
120432    4
120433    5
120434    6
Length: 120435, dtype: int64
0         8
1         7
2         6
3         5
4         4
         ..
120430    4
120431    3
120432    2
120433    1
120434    0
Length: 120435, dtype: int64


In [ ]:
#Cosine similarity with question
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count_vectorizer=CountVectorizer()
for i in range(samples):
  count_matrix=count_vectorizer.fit_transform([preprocessString(questions_answers['QBody'][i]),preprocessString(questions_answers['Body'][i])])
  cs[i]=cosine_similarity(count_matrix[0],count_matrix[1])
print(cs)

In [ ]:
#Average cosine similarity with other


**KARTHIK FEATURE SELECTION**

In [ ]:
# User history
# Cumulative scores/answers/questions
user_answer_ids = questions_answers.groupby('OwnerUserId')
print(user_answer_ids)
user_cumulative_score = questions_answers.groupby('OwnerUserId', sort=False).agg({'Score': 'sum'})
user_cumulative_answers = questions_answers.groupby('OwnerUserId', sort=False).size()
user_cumulative_questions = questions_answers.groupby('QAskerId', sort=False).size()

#print(user_cumulative_answers)
user_stats = pd.DataFrame(user_cumulative_score)
user_stats['Answers'] = user_cumulative_answers

from datetime import datetime as dt
import time

def toYearFraction(date):
    
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = dt(year=year, month=1, day=1)
    startOfNextYear = dt(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

# Account age
format = "%Y-%m-%dT%H:%M:%SZ"

today = dt.today()
today = toYearFraction(today)
print(today)

creation_date = questions_answers['CreationDate'].astype(str)
creation_date = pd.to_datetime(creation_date)
creation_date = creation_date.apply(toYearFraction)

creation_date = today - creation_date
print(creation_date)

'''print(user_cumulative_score)
print(user_cumulative_answers)
print(user_cumulative_questions)'''

2020.9281965315727
0         12.343997
1         12.341417
2         12.341327
3         12.229312
4         12.213258
            ...    
120430     4.128783
120431     4.128777
120432     4.128751
120433     4.128741
120434     4.128740
Name: CreationDate, Length: 120435, dtype: float64


In [ ]:
# Average user score (for their answers)
def find_user_average_score(scores, num_answers):
  return lambda row: 0.0 if row[num_answers] == 0 else float(row[scores]/row[num_answers])

user_stats.loc[:, 'Average_Score'] = user_stats.apply(find_user_average_score('Score', 'Answers'), axis=1)      

In [ ]:
print(user_stats['Score'])
print(type(user_stats))

OwnerUserId
50.0          111
86.0           14
157.0        1685
5845.0         34
16632.0       165
             ... 
5820675.0       3
7020477.0       0
5547818.0       0
4989385.0       0
649776.0        0
Name: Score, Length: 69775, dtype: int64
<class 'pandas.core.frame.DataFrame'>


In [ ]:
questions_answers.groupby(['OwnerUserId', 'QAskerId']).ngroups

113205

In [ ]:
questions_answers.to_csv()

**FUNCTIONS**

In [ ]:
import re
from bs4 import BeautifulSoup
import string
#Remove urls, html tags, code, and punctuation from strings
def preprocessString(text):
  text=text.lower()
  #Remove HTML Tags and Code
  soup=BeautifulSoup(text)
  for code in soup.find_all('code'):
    code.extract()
  text=soup.get_text()
  #Remove URLS
  text=re.sub('http://\S+|https://\S+', '', text)
  #Remove punctuation
  text=text.translate(str.maketrans('', '', string.punctuation))
  #Join lines together
  text=" ".join(line.strip() for line in text.splitlines())
  return text.strip()